In [0]:
from pyspark.sql import SparkSession, functions as f

In [0]:
sc= SparkSession.builder.appName('world cup').getOrCreate()

In [0]:
df=sc.read.csv('/FileStore/tables/wc2018_players.csv', header=True, inferSchema=True)

In [0]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [0]:
df.createOrReplaceTempView('wc')

In [0]:
#1
sc.sql('select `FIFA Popular Name`, Height+1 from wc').show(5)

+------------------+------------+
| FIFA Popular Name|(Height + 1)|
+------------------+------------+
|TAGLIAFICO Nicolas|         170|
|    PAVON Cristian|         170|
|    LANZINI Manuel|         168|
|    SALVIO Eduardo|         168|
|      MESSI Lionel|         171|
+------------------+------------+
only showing top 5 rows



In [0]:
#2
df.select('FIFA Popular Name', f.when(df.Height>170, 'tall').when(df.Height<170, 'short').otherwise(170)).show(5)

+------------------+------------------------------------------------------------------------------+
| FIFA Popular Name|CASE WHEN (Height > 170) THEN tall WHEN (Height < 170) THEN short ELSE 170 END|
+------------------+------------------------------------------------------------------------------+
|TAGLIAFICO Nicolas|                                                                         short|
|    PAVON Cristian|                                                                         short|
|    LANZINI Manuel|                                                                         short|
|    SALVIO Eduardo|                                                                         short|
|      MESSI Lionel|                                                                           170|
+------------------+------------------------------------------------------------------------------+
only showing top 5 rows



In [0]:
#3
df.select('FIFA Popular Name', f.when(df.Height>170, 1).when(df.Height<170, 0).otherwise(.01)).show(5)

+------------------+------------------------------------------------------------------------+
| FIFA Popular Name|CASE WHEN (Height > 170) THEN 1 WHEN (Height < 170) THEN 0 ELSE 0.01 END|
+------------------+------------------------------------------------------------------------+
|TAGLIAFICO Nicolas|                                                                     0.0|
|    PAVON Cristian|                                                                     0.0|
|    LANZINI Manuel|                                                                     0.0|
|    SALVIO Eduardo|                                                                     0.0|
|      MESSI Lionel|                                                                    0.01|
+------------------+------------------------------------------------------------------------+
only showing top 5 rows



In [0]:
#4
df.orderBy('Height').select('FIFA Popular Name').show(1)

+-----------------+
|FIFA Popular Name|
+-----------------+
|   YAHIA ALSHEHRI|
+-----------------+
only showing top 1 row



In [0]:
#5
df.orderBy(df.Height.desc()).show()

+--------------+---+----+------------------+----------+-----------+--------------------+------+------+
|          Team|  #|Pos.| FIFA Popular Name|Birth Date| Shirt Name|                Club|Height|Weight|
+--------------+---+----+------------------+----------+-----------+--------------------+------+------+
|       Croatia| 12|  GK|     KALINIC Lovre|03.04.1990| L. KALINIĆ|      KAA Gent (BEL)|   201|    96|
|       Denmark|  3|  DF|VESTERGAARD Jannik|03.08.1992|VESTERGAARD|VfL Borussia Mönc...|   200|    98|
|     Argentina|  6|  DF|    FAZIO Federico|17.03.1987|      FAZIO|       AS Roma (ITA)|   199|    85|
|       Belgium|  1|  GK|  COURTOIS Thibaut|11.05.1992|   COURTOIS|    Chelsea FC (ENG)|   199|    91|
|       Iceland| 12|  GK|   SCHRAM Frederik|19.01.1995|     SCHRAM|   Roskilde BK (DEN)|   198|    92|
|        Sweden|  1|  GK|       OLSEN Robin|08.01.1990|      OLSEN|  FC Kobenhavn (DEN)|   198|    89|
|       Belgium| 13|  GK|     CASTEELS Koen|25.06.1992|   CASTEELS| VfL W

In [0]:
df.select('FIFA Popular Name').filter(df.Height==201).show()

+-----------------+
|FIFA Popular Name|
+-----------------+
|    KALINIC Lovre|
+-----------------+



In [0]:
#6
df.select(avg('Height')).show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-3955293502855212> in <module>
      1 #6
----> 2 df.select(avg('Height')).show()

NameError: name 'avg' is not defined

In [0]:
mov= sc.read.csv('/FileStore/tables/movies.csv', header=True, inferSchema=True)

In [0]:
rat=sc.read.csv('dbfs:/FileStore/tables/ratings.csv', header=True, inferSchema=True)

In [0]:
mov.show(10)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



In [0]:
df2=mov.join(rat, on ='movieId', how='outer')

In [0]:
df2.createOrReplaceTempView('movie')

In [0]:
sc.sql('select * from movie').show()

+-------+----------------+--------------------+------+------+----------+
|movieId|           title|              genres|userId|rating| timestamp|
+-------+----------------+--------------------+------+------+----------+
|      1|Toy Story (1995)|Adventure|Animati...|     7|   3.0| 851866703|
|      1|Toy Story (1995)|Adventure|Animati...|     9|   4.0| 938629179|
|      1|Toy Story (1995)|Adventure|Animati...|    13|   5.0|1331380058|
|      1|Toy Story (1995)|Adventure|Animati...|    15|   2.0| 997938310|
|      1|Toy Story (1995)|Adventure|Animati...|    19|   3.0| 855190091|
|      1|Toy Story (1995)|Adventure|Animati...|    20|   3.5|1238729767|
|      1|Toy Story (1995)|Adventure|Animati...|    23|   3.0|1148729853|
|      1|Toy Story (1995)|Adventure|Animati...|    26|   5.0|1360087980|
|      1|Toy Story (1995)|Adventure|Animati...|    30|   4.0| 944943070|
|      1|Toy Story (1995)|Adventure|Animati...|    37|   4.0| 981308121|
|      1|Toy Story (1995)|Adventure|Animati...|    

In [0]:
sc.sql('select rating, count(rating) as rating_count from movie group by rating').show()

+------+------------+
|rating|rating_count|
+------+------------+
|   3.5|       10538|
|   4.5|        7723|
|  null|           0|
|   2.5|        4449|
|   1.0|        3326|
|   4.0|       28750|
|   0.5|        1101|
|   3.0|       20064|
|   2.0|        7271|
|   1.5|        1687|
|   5.0|       15095|
+------+------------+



In [0]:
sc.sql('select userId, count(userId) as user_count from movie group by userId').show()

+------+----------+
|userId|user_count|
+------+----------+
|   463|       483|
|   471|       216|
|   496|       126|
|   148|       132|
|   623|       103|
|   243|       307|
|   392|        25|
|   540|        20|
|   516|       149|
|    31|        69|
|   580|       922|
|    85|       107|
|   137|        80|
|   451|        52|
|   251|       119|
|   458|        76|
|    65|        27|
|   481|       436|
|   588|        52|
|   255|       145|
+------+----------+
only showing top 20 rows



In [0]:
sc.sql('select title, sum(rating) from movie group by title').show()

+--------------------+-----------+
|               title|sum(rating)|
+--------------------+-----------+
|    Fair Game (1995)|       17.0|
| If Lucy Fell (1996)|       14.0|
| Three Wishes (1995)|        4.0|
|Heavenly Creature...|      157.0|
|Paris, France (1993)|        5.0|
|Snow White and th...|      260.0|
|         Cosi (1996)|        8.5|
|Night of the Livi...|      111.0|
|When We Were King...|       71.0|
|       Psycho (1960)|      327.5|
|   Annie Hall (1977)|      327.5|
|Men in Black (a.k...|      696.0|
|Odd Couple II, Th...|        6.0|
|In the Heat of th...|       86.0|
|Seven Beauties (P...|       10.0|
|    Elizabeth (1998)|      179.0|
|First Blood (Ramb...|       62.5|
|Problem Child (1990)|       10.0|
|Gulliver's Travel...|       11.0|
|7th Voyage of Sin...|       29.0|
+--------------------+-----------+
only showing top 20 rows



In [0]:
sc.sql('select title, avg(rating) from movie group by title').show()

+--------------------+------------------+
|               title|       avg(rating)|
+--------------------+------------------+
|    Fair Game (1995)|2.4285714285714284|
| If Lucy Fell (1996)|               2.8|
| Three Wishes (1995)|               4.0|
|Heavenly Creature...|3.6511627906976742|
|Paris, France (1993)|               5.0|
|Snow White and th...|3.6619718309859155|
|         Cosi (1996)|2.8333333333333335|
|Night of the Livi...| 4.111111111111111|
|When We Were King...|            4.4375|
|       Psycho (1960)| 4.253246753246753|
|   Annie Hall (1977)|           4.09375|
|Men in Black (a.k...| 3.663157894736842|
|Odd Couple II, Th...|               2.0|
|In the Heat of th...| 3.909090909090909|
|Seven Beauties (P...|3.3333333333333335|
|    Elizabeth (1998)|3.7291666666666665|
|First Blood (Ramb...|3.4722222222222223|
|Problem Child (1990)|               2.5|
|Gulliver's Travel...|3.6666666666666665|
|7th Voyage of Sin...| 4.142857142857143|
+--------------------+------------